## Installation des librairies

In [ ]:
!pip install elasticsearch

In [ ]:
!pip install elasticsearch-dsl

## Importation des librairies

In [ ]:
import pandas as pd
import json
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)

## Connexion à Elasticsearch

In [ ]:
# @title **Connexion**

es = Elasticsearch(
    hosts=['http://formakuntza-kibana.hupi.io:9200/'],
    http_auth=('student', 'gCzOD2pVnCqVpd13')
)

try:
    # Récupérer les informations du cluster
    info = es.info()

    # Afficher des informations intéressantes
    print(f"Connexion réussie au cluster : {info['cluster_name']}")
    print(f"Version d'Elasticsearch : {info['version']['number']}")

    # Afficher les indices existants
    indices = es.cat.indices(format="json")
    print(f"Nombre d'index dans le cluster : {len(indices)}")

except Exception as e:
    print(f"Erreur de connexion : {e}")

<ipython-input-11-32068d401aa2>:3: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


Connexion réussie au cluster : docker-cluster
Version d'Elasticsearch : 7.17.24
Nombre d'index dans le cluster : 11


In [ ]:
# @title **Afficher les index présents**
# Obtenir les détails de tous les index
indices = es.cat.indices(format="json")

# Afficher les détails des index
for index in indices:
    print(f"Nom de l'index: {index['index']}")
    print(f"État: {index['status']}")
    print(f"Taille en disque: {index['store.size']}")
    print(f"Nombre de documents: {index['docs.count']}")
    print("-" * 50)

Nom de l'index: .kibana_7.17.24_001
État: open
Taille en disque: 12.8mb
Nombre de documents: 3412
--------------------------------------------------
Nom de l'index: .geoip_databases
État: open
Taille en disque: 33.6mb
Nombre de documents: 35
--------------------------------------------------
Nom de l'index: .security-7
État: open
Taille en disque: 263.4kb
Nombre de documents: 53
--------------------------------------------------
Nom de l'index: books
État: open
Taille en disque: 5.4mb
Nombre de documents: 6810
--------------------------------------------------
Nom de l'index: .apm-custom-link
État: open
Taille en disque: 227b
Nombre de documents: 0
--------------------------------------------------
Nom de l'index: ecommerce
État: open
Taille en disque: 613kb
Nombre de documents: 4675
--------------------------------------------------
Nom de l'index: .apm-agent-configuration
État: open
Taille en disque: 227b
Nombre de documents: 0
--------------------------------------------------
Nom d

In [ ]:
# @title **Afficher l'index "ecommerce"**
index_name = "ecommerce"

s = Search(using=es, index=index_name).query("match_all")

# Utilisation de la méthode scan pour récupérer tous les résultats rapidement
results = s.scan()

# Convertir les résultats en DataFrame
data = [hit.to_dict() for hit in results]
df = pd.json_normalize(data)
print(f"Show dataframe shape: {df.shape}")
print(f"Show dataframe first 10 rows: \n {df.head()}")

Show dataframe shape: (4676, 11)
Show dataframe first 10 rows: 
                    product geoip_city_name geoip_continent_name                 order_date                @timestamp  total_quantity currency  \
0       Ballon de football       Marrakesh               Africa  2025-01-05T07:39:22+00:00  2025-01-05T07:39:22.000Z               2      EUR   
1       Rouleau de massage      Bogotu00e1        South America  2025-01-05T08:22:34+00:00  2025-01-05T08:22:34.000Z               2      EUR   
2   Chaussures de football           Cairo               Africa  2025-01-05T01:48:00+00:00  2025-01-05T01:48:00.000Z               2      EUR   
3          Gourde de sport       Abu Dhabi                 Asia  2025-01-12T18:11:12+00:00  2025-01-12T18:11:12.000Z               1      EUR   
4  Chaussures de randonnée        New York        North America  2025-01-01T00:05:46+00:00  2025-01-01T00:05:46.000Z               1      EUR   

   order_id customer_gender day_of_week  taxful_total_price  
0 



---


# Exercice 1 : `Indexation`

In [ ]:
###############################################   # Exercice 1: Indexation

# Énoncé: Indexez un nouveau document dans l'index ecommerce.

doc = # To complete : use a dictionnary

response = # To complete : use Elasticsearch "index" method
print(response)



---


# Exercice 2 : `Recherche simple`

In [ ]:
###############################################   Exercice 2: Recherche simple

## Énoncé: Effectuez une recherche pour tous les produits dont le nom contient le mot "sport".

response =  # To complete : use Elasticsearch search method with match query

print(response)

In [ ]:
# Obtenir la clé « hits » et comprendre le modèle
hits = response['hits']['hits']
hits

[{'_shard': '[staging_ecommerce][0]',
  '_node': 'w4J3bTfdQgWTrMMKXDMARw',
  '_index': 'staging_ecommerce',
  '_type': '_doc',
  '_id': '588CTZQB5IuiTNqkwr6B',
  '_score': 3.6267476,
  '_source': {'product': 'Chaussures de randonnée',
   'geoip_city_name': 'New York',
   'geoip_continent_name': 'North America',
   'order_date': '2024-12-30T11:58:34+00:00',
   '@timestamp': '2024-12-30T11:58:34.000Z',
   'total_quantity': 4,
   'currency': 'EUR',
   'order_id': 729807,
   'customer_gender': 'FEMALE',
   'day_of_week': 'Monday',
   'taxful_total_price': 117.96},
  '_explanation': {'value': 3.6267476,
   'description': 'weight(product:Chaussures de randonnée in 29) [PerFieldSimilarity], result of:',
   'details': [{'value': 3.6267476,
     'description': 'score(freq=1.0), computed as boost * idf * tf from:',
     'details': [{'value': 2.2, 'description': 'boost', 'details': []},
      {'value': 3.6267476,
       'description': 'idf, computed as log(1 + (N - n + 0.5) / (n + 0.5)) from:',
 

In [ ]:
# Obtenir les informations sur la source (les données)
source_list = [hit['_source'] for hit in hits]
df = pd.json_normalize(source_list)
print(df)

                   product geoip_city_name geoip_continent_name                 order_date                @timestamp  total_quantity currency  \
0  Chaussures de randonnée        New York        North America  2024-12-30T11:58:34+00:00  2024-12-30T11:58:34.000Z               4      EUR   
1  Chaussures de randonnée      Birmingham               Europe  2025-01-13T16:53:46+00:00  2025-01-13T16:53:46.000Z               2      EUR   
2  Chaussures de randonnée          Cannes               Europe  2025-01-19T16:29:17+00:00  2025-01-19T16:29:17.000Z               2      EUR   
3  Chaussures de randonnée       Abu Dhabi                 Asia  2024-12-30T19:56:38+00:00  2024-12-30T19:56:38.000Z               2      EUR   
4  Chaussures de randonnée      Birmingham               Europe  2025-01-21T23:49:55+00:00  2025-01-21T23:49:55.000Z               2      EUR   
5  Chaussures de randonnée             NaN               Europe  2025-01-20T23:24:00+00:00  2025-01-20T23:24:00.000Z              



---


# Exercice 3 : `Mise à jour`

In [ ]:
###############################################   Exercice 3: Mise à jour

# Énoncé: Mettez à jour le document que vous avez indexé précédemment
# (utilisez l'ID du document retourné lors de l'indexation) pour changer la valeur de la colonne 'quantity' par 95.

response =  # To complete : use Elasticsearch update method en spécifiant correctement l'id par votre id du document

print(response)

{'_index': 'staging_ecommerce', '_type': '_doc', '_id': 'mc8OTZQB5IuiTNqkvdzi', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4679, '_primary_term': 1}


In [ ]:
# Vérifier le remplacement avec la fonction "match"
response =

print(response)

{'took': 112, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 4, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_shard': '[staging_ecommerce][0]', '_node': 'w4J3bTfdQgWTrMMKXDMARw', '_index': 'staging_ecommerce', '_type': '_doc', '_id': 'mc8OTZQB5IuiTNqkvdzi', '_score': 1.0, '_source': {'product': 'Chaussures de sport', 'geoip_city_name': 'Pau', 'geoip_continent_name': 'Europe', 'order_date': '2025-01-09T14:04:19+00:00', '@timestamp': '2025-01-09T14:04:19.000Z', 'total_quantity': 95, 'currency': 'Europe', 'order_id': 670552, 'customer_gender': 'MALE', 'day_of_week': 'Thursday', 'taxful_total_price': 89.99}, '_explanation': {'value': 1.0, 'description': 'order_id:[670552 TO 670552]', 'details': []}}, {'_shard': '[staging_ecommerce][0]', '_node': 'w4J3bTfdQgWTrMMKXDMARw', '_index': 'staging_ecommerce', '_type': '_doc', '_id': 'ls8JTZQB5IuiTNqkU9xQ', '_score': 1.0, '_source': {'product': 'Chaussures de sport', 'geoip_ci

In [ ]:
# Obtenir les informations sur la source (les données)
hits = response['hits']['hits']
source_list = [hit['_source'] for hit in hits]
df = pd.json_normalize(source_list)
print(df)

               product geoip_city_name geoip_continent_name                 order_date                @timestamp  total_quantity currency  order_id  \
0  Chaussures de sport             Pau               Europe  2025-01-09T14:04:19+00:00  2025-01-09T14:04:19.000Z              95   Europe    670552   
1  Chaussures de sport             Pau               Europe  2025-01-09T14:04:19+00:00                       NaN               3   Europe    670552   
2  Chaussures de sport             Pau               Europe  2025-01-09T14:04:19+00:00                       NaN               3   Europe    670552   
3  Chaussures de sport             Pau               Europe  2025-01-09T14:04:19+00:00                       NaN               3   Europe    670552   

  customer_gender day_of_week  taxful_total_price  
0            MALE    Thursday               89.99  
1            MALE    Thursday               89.99  
2            MALE    Thursday               89.99  
3            MALE    Thursday       



---


# Exercice 4 : `Agrégations`

In [ ]:
###############################################   Exercice 4: Agrégations

# Énoncé: Obtenez le prix moyen de tous les produits dans l'index ecommerce.

# Correction:
response = # To complete : use Elasticsearch search method with aggs function

avg_price = response['aggregations']['avg_price']['value']
print(f"Prix moyen: {avg_price} EUR")


Prix moyen: 75.06283393887583 EUR




---


# Exercice 5 : `Suppression`

In [ ]:
####################################################    Exercice 5: Suppression
##Correction: (Remplacez YOUR_DOCUMENT_ID par l'ID réel)

response = # To complete : use Elasticsearch delete method
print(response)

{'_index': 'staging_ecommerce', '_type': '_doc', '_id': 'mc8OTZQB5IuiTNqkvdzi', '_version': 3, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4680, '_primary_term': 1}




---


# Exercice 7 : `Recherche multiple`

In [ ]:
###############################################   Exercice 7: Recherche multiple

## Énoncé: Effectuez une recherche pour tous les produits dont le nom contient le mot "sport".

response = # To complete : use Elasticsearch search method with multi_match function
print(response)

{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 124, 'relation': 'eq'}, 'max_score': 3.6269612, 'hits': [{'_shard': '[staging_ecommerce][0]', '_node': 'w4J3bTfdQgWTrMMKXDMARw', '_index': 'staging_ecommerce', '_type': '_doc', '_id': '588CTZQB5IuiTNqkwr6B', '_score': 3.6269612, '_source': {'product': 'Chaussures de randonnée', 'geoip_city_name': 'New York', 'geoip_continent_name': 'North America', 'order_date': '2024-12-30T11:58:34+00:00', '@timestamp': '2024-12-30T11:58:34.000Z', 'total_quantity': 4, 'currency': 'EUR', 'order_id': 729807, 'customer_gender': 'FEMALE', 'day_of_week': 'Monday', 'taxful_total_price': 117.96}, '_explanation': {'value': 3.6269612, 'description': 'weight(product:Chaussures de randonnée in 29) [PerFieldSimilarity], result of:', 'details': [{'value': 3.6269612, 'description': 'score(freq=1.0), computed as boost * idf * tf from:', 'details': [{'value': 2.2, 'description': 'boost', 'd

In [ ]:
# Obtenir la clé « hits » et comprendre le modèle
hits = response['hits']['hits']
hits

[{'_shard': '[staging_ecommerce][0]',
  '_node': 'w4J3bTfdQgWTrMMKXDMARw',
  '_index': 'staging_ecommerce',
  '_type': '_doc',
  '_id': '588CTZQB5IuiTNqkwr6B',
  '_score': 3.6269612,
  '_source': {'product': 'Chaussures de randonnée',
   'geoip_city_name': 'New York',
   'geoip_continent_name': 'North America',
   'order_date': '2024-12-30T11:58:34+00:00',
   '@timestamp': '2024-12-30T11:58:34.000Z',
   'total_quantity': 4,
   'currency': 'EUR',
   'order_id': 729807,
   'customer_gender': 'FEMALE',
   'day_of_week': 'Monday',
   'taxful_total_price': 117.96},
  '_explanation': {'value': 3.6269612,
   'description': 'weight(product:Chaussures de randonnée in 29) [PerFieldSimilarity], result of:',
   'details': [{'value': 3.6269612,
     'description': 'score(freq=1.0), computed as boost * idf * tf from:',
     'details': [{'value': 2.2, 'description': 'boost', 'details': []},
      {'value': 3.6269612,
       'description': 'idf, computed as log(1 + (N - n + 0.5) / (n + 0.5)) from:',
 

In [ ]:
# Obtenir les informations sur la source (les données)
source_list = [hit['_source'] for hit in hits]
df = pd.json_normalize(source_list)
print(df)

                   product geoip_city_name geoip_continent_name                 order_date                @timestamp  total_quantity currency  \
0  Chaussures de randonnée        New York        North America  2024-12-30T11:58:34+00:00  2024-12-30T11:58:34.000Z               4      EUR   
1  Chaussures de randonnée      Birmingham               Europe  2025-01-13T16:53:46+00:00  2025-01-13T16:53:46.000Z               2      EUR   
2  Chaussures de randonnée          Cannes               Europe  2025-01-19T16:29:17+00:00  2025-01-19T16:29:17.000Z               2      EUR   
3  Chaussures de randonnée       Abu Dhabi                 Asia  2024-12-30T19:56:38+00:00  2024-12-30T19:56:38.000Z               2      EUR   
4  Chaussures de randonnée      Birmingham               Europe  2025-01-21T23:49:55+00:00  2025-01-21T23:49:55.000Z               2      EUR   
5  Chaussures de randonnée             NaN               Europe  2025-01-20T23:24:00+00:00  2025-01-20T23:24:00.000Z              